In [ ]:
!pip install azure.eventhub

In [ ]:
import asyncio
from azure.eventhub.aio import EventHubProducerClient
from azure.eventhub import EventData
import json

In [ ]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
connection_str = '<Connection string-primary key>' # found under Shared Access Policies 
consumer_group = '<Consumer Group Name>'
eventhub_name = '<EventHub Name>'

In [ ]:
def readjson():
    with open("StormEvents.json", "r") as read_file:
        data = json.load(read_file)
    return data

In [ ]:
async def create_batch(client):
    event_data_batch = await client.create_batch()
    data = readjson()
    can_add = True
    for i in range(10):
        try:
            event_data_batch.add(EventData(str(data[i])))
        except ValueError:
            can_add = False  # EventDataBatch object reaches max_size.
    return event_data_batch

In [ ]:
async def send():
    client = EventHubProducerClient.from_connection_string(connection_str, eventhub_name=eventhub_name)
    batch_data = await create_batch(client)
    async with client:
        await client.send_batch(batch_data)

In [ ]:
loop = asyncio.get_event_loop()
loop.run_until_complete(send())